In [1]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks, peak_prominences
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [2]:
dg_df = pd.read_csv('./big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/Demographics.csv')
dg_df.head(16)

,ID,Gender,HbA1c
0,13,MALE,5.7
1,1,FEMALE,5.5
2,3,FEMALE,5.9
3,4,FEMALE,6.4
4,5,FEMALE,5.7
5,2,MALE,5.6
6,6,FEMALE,5.8
7,7,FEMALE,5.3
8,8,FEMALE,5.6
9,10,FEMALE,6.0


## ID별 CGM 결측치 확인

In [7]:
for id in range(1, len(dg_df)+1):
    if id == 15: # Skip ID 15
        continue
    dexcom_df = pd.read_csv(f'./big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/{id:03}/Dexcom_{id:03}.csv')

    biological_sex = dg_df.loc[dg_df['ID'] == id, 'Gender'].values[0]
    hbA1c = dg_df.loc[dg_df['ID'] == id, 'HbA1c'].values[0]

    print(f'[ID: {id}] Now processing ...')

    ## Dexcom Glucose
    dexcom_df.drop(dexcom_df.loc[:11].index, inplace=True)
    dexcom_df.rename(columns = {'Timestamp (YYYY-MM-DDThh:mm:ss)': 'datetime'}, inplace = True)
    dexcom_df.rename(columns = {'Glucose Value (mg/dL)': 'glucose'}, inplace = True)
    dexcom_df = dexcom_df[['datetime', 'glucose']]
    dexcom_df['datetime']=pd.to_datetime(dexcom_df['datetime'])
    dexcom_df.set_index('datetime', inplace=True)

    # 시간 간격 계산
    dexcom_df['time_diff'] = dexcom_df.index.to_series().diff().dt.total_seconds() / 60.0

    # 4분에서 6분 사이가 아닌 시간 간격을 찾기
    missing_intervals = dexcom_df[(dexcom_df['time_diff'] > 6) | (dexcom_df['time_diff'] < 4)]

    # 결과 출력
    print(f"ID {id} 결측치 구간:")
    print(missing_intervals)

[ID: 1] Now processing ...
ID 1 결측치 구간:
                     glucose   time_diff
datetime                                
2020-02-15 09:13:30     83.0   10.000000
2020-02-20 02:48:25    112.0  189.983333
[ID: 2] Now processing ...
ID 2 결측치 구간:
                     glucose   time_diff
datetime                                
2020-02-25 12:43:30    118.0  844.983333
[ID: 3] Now processing ...
ID 3 결측치 구간:
                     glucose  time_diff
datetime                               
2020-02-25 11:21:40     93.0       65.0
[ID: 4] Now processing ...
ID 4 결측치 구간:
                     glucose   time_diff
datetime                                
2020-02-29 03:41:16     98.0   14.966667
2020-03-03 05:11:11    129.0  459.983333
[ID: 5] Now processing ...
ID 5 결측치 구간:
                     glucose  time_diff
datetime                               
2020-03-04 01:06:16     98.0       76.4
[ID: 6] Now processing ...
ID 6 결측치 구간:
Empty DataFrame
Columns: [glucose, time_diff]
Index: []
[ID: 7] Now p